In [ ]:
import math
import numpy as np
import pandas as pd

from math import sin, cos, log, log10, exp, pi
from typing import Callable, Optional


pd.set_option("display.width", 200)

functions = [
    lambda x: x**2 + log(x), lambda x: x**2 - log10(x + 2), lambda x: x**2 + log(x) - 4,
    lambda x: (x - 1)**2 - 0.5*exp(x), lambda x: (x - 1)**2 - exp(-x), lambda x: x**3 - sin(x),
    lambda x: x - cos(x), lambda x: x**2 - cos(pi * x), lambda x: x**2 - sin(pi * x),
    lambda x: x**2 - cos(0.5 * pi * x), lambda x: x - 2*cos(0.5 * pi * x),
    lambda x: x - sin(pi * x), lambda x: 2*x - cos(x), lambda x: x**2 + log(x + 5),
    lambda x: 0.5 * x**2 + cos(2 * x), lambda x: x**2 - 0.5*exp(-x),
    lambda x: x**2 + log10(x), lambda x: x - log10(x + 2), lambda x: x**2 - log10(0.5 * x),
    lambda x: x**3 - cos(2 * x), lambda x: x**2 + cos(pi * x / 2), lambda x: x/2 - cos(x / 2)
]

values = [
    [0.52, 0.42, 0.87, 0.67],
    [0.53, 0.52, 0.97, 0.73],
    [1.52, 1.52, 1.97, 1.77],
    [0.13, 0.12, 0.57, 0.33],
    [1.07, 1.02, 1.47, 1.27],
    [0.92, 0.62, 1.07, 0.83],
    [0.37, 0.12, 0.57, 0.37],
    [0.77, 0.52, 0.97, 0.73],
    [0.92, 0.53, 0.98, 0.77],
    [0.37, 0.12, 0.58, 0.33],
    [0.53, 0.43, 0.86, 0.67],
    [0.64, 0.42, 0.87, 0.63],
    [0.71, 0.43, 0.87, 0.67],
    [0.88, 0.63, 1.08, 0.83],
    [0.44, 0.13, 0.58, 0.37],
    [0.73, 0.52, 0.97, 0.73],
    [0.84, 0.62, 1.07, 0.83],
    [0.37, 0.12, 0.58, 0.33],
    [0.53, 0.43, 0.86, 0.67],
    [0.77, 0.52, 0.97, 0.73],
    [0.92, 0.53, 0.98, 0.77],
    [0.37, 0.12, 0.58, 0.33],
    [0.13, 0.12, 0.57, 0.33],
    [0.64, 0.42, 0.87, 0.63],
]

intervals = [
    [0.4, 0.9], [0.5, 1.0], [1.5, 2.0], [0.1, 0.6], [1.0, 1.5], [0.6, 1.1],
    [0.1, 0.6], [0.5, 1.0], [0.5, 1.0], [0.1, 0.6], [0.4, 0.9], [0.4, 0.9],
    [0.4, 0.9], [0.6, 1.1], [0.1, 0.6], [0.5, 1.0], [0.6, 1.1], [0.1, 0.6],
    [0.4, 0.9], [0.5, 1.0], [0.5, 1.0], [0.1, 0.6], [0.1, 0.6], [0.4, 0.9]
]

abs_error = lambda x, y: abs(x - y) if x else "inf"


def compute_divided_differences_table(grid_nodes: list[int | float],
                                      grid_values: list[int | float]) -> list[float]:
    """
    Вычисляет таблицу разделенных разностей и возвращает разделенные разности.

    Args:
        grid_nodes (list[int | float]): Узловые точки (x_i).
        grid_values (list[int | float]): Значения функции в узловых точках (f(x_i)).

    Returns:
        list[float]: Список разделенных разностей.
    """
    n = len(grid_nodes)
    if n == 0:
        return []
    if n != len(grid_values):
        raise ValueError("grid_nodes и grid_values должны быть одной длины.")

    table = np.zeros((n, n))
    table[:, 0] = grid_values 

    for j in range(1, n): 
        for i in range(n - j):
            numerator = table[i+1, j-1] - table[i, j-1]
            denominator = grid_nodes[i+j] - grid_nodes[i]
            table[i, j] = numerator / denominator

    return table[0, :].tolist()


def interpolate(function: Callable[[int | float], float],
                x: int | float, a: int = 0, b: int = 0, 
                count: int = 10,
                grid: Optional[list[int | float]] = None,
                is_Lagrange: bool = True) -> Optional[float]:
    """
    Возвращает приближенное значение заданной функции в точке `x` при помощи интерпалиционного многочлена Лагранжа/Ньютона с равномерной сеткой.
    
    Args:
        function (Callable[[int | float], float]): функция, которую необходимо интерполировать.
        x (int | float): точка в которой нужно вычислить значение функции.
        a (int): значение первого узла (default 0).
        b (int): значение последнего узла (defautl 0).
        grid (Optional[list[int | float]]): если None, то сетка создается с равномерным шагом от `a` до `b` (и количество узлов равно count + 1) (default None).
        count (int): кол-во узлов сетки + 1 (default 10). 
        is_Lagrange (bool): True - если используется полином Лагранжа, иначе Ньютона (default True).
    
    Returns:
        float: вычесленной значение в точке `x`. 
        None: если нельзя вычислить значение функции в узле.
    """
    if grid is None:
        step = (b - a)/count
        grid = [a + i*step for i in range(count + 1)]
    else:
        grid = list(set(grid))
        count = len(grid) - 1
        
    y_values = []
    for node in grid:
        try:
            y_values.append(function(node))
        except (ValueError, TypeError, ZeroDivisionError) as e:
            print(f"Невозможно вычислить значение функции в узле `{node}`. Ошибка: {e}")
            return None
    
    if is_Lagrange:
        y_interpolate = 0
        
        for i in range(count + 1):
            prod = 1
            
            for j in range(count + 1):
                if i == j: continue
                prod *= (x - grid[j])/(grid[i] - grid[j])
            
            y_interpolate += prod * y_values[i]
    else:
        coeffs = compute_divided_differences_table(grid, y_values)
        y_interpolate, prod = 0, 1.0
        
        for i in range(count + 1):
            y_interpolate += coeffs[i] * prod
            prod *= (x - grid[i])         
        
    return y_interpolate
        

def get_k_nearest_nodes(grid: list[int | float], 
                        x: int | float, k: int = 1) -> list[int | float]:
    """
    Возвращает `k` ближайщих к `x` узлов в заданной сетке.
    
    Args:
        grid (list[int | float]): сетка узлов.
        x (int | float): значение по отнощению к которому будут искаться ближайщие узлы.
        k (int): количество ближайщих узлов (default 1).
    
    Returns:
        list[int | float]: ближайщие к `x` узлы сетки `grid`.
    """
    k = max(1, k)
    if k > len(grid): 
        k = len(grid) - 1
        
    return sorted(grid, key=lambda y: abs(x - y))[:k]
    
        
def get_interpolation_error_bounds(function: Callable[[int | float], float], 
                                   derivative_function: Callable[[int | float], float],
                                   min_derivative: Callable[[int | float], float], max_derivative: Callable[[int | float], float],
                                   grid: list[int | float], 
                                   x: int | float,
                                   n: int = 1) -> tuple[float]:
    """
    Возвращает нижнюю и верхнюю границы ошибки интерполирования и приближенное значение заданной функции в точке `x`.
    
    Args: 
        function (Callable[[int | float], float]): функция для которой необходимо вычислить интервал.
        derivative_function (Callable[[int | float], float]): производная n + 1 порядка исходной функции.
        min_derivative Callable[[int | float], float]: функция, которая определяет точку в которой производная имеет наименьшее значение.
        max_derivative Callable[[int | float], float]: функция, которая определяет точку в которой производная имеет наибольшее значение.
        grid (list[int | float]): сетка интерполяции.
        x (int | float): точка в которой необходимо вычислить интервал.
        n (int): порядок интерполирующего полинома (default 1).
    
    Returns:
        tuple[float]: граница интервала ошибки интерполяции и приближенное значение заданной функции в точке `x`, посчитаное по Лагранжу и Ньютону.
    """
    nearest_nodes = get_k_nearest_nodes(grid, x, k=n+1)
    derivative_min, derivative_max = derivative_function(min_derivative(nearest_nodes)), derivative_function(max_derivative(nearest_nodes))
    
    factorial = math.factorial(n + 1)
    prod = 1 / factorial
    for value in nearest_nodes:
        prod *= (x - value)
        
    derivative_min, derivative_max = sorted([derivative_min * prod, derivative_max * prod])
    return derivative_min, derivative_max, interpolate(function, x, grid=nearest_nodes), interpolate(function, x, grid=nearest_nodes, is_Lagrange=False)


def test_uniform_grid() -> None:
    """Тестирование интерполяции Лагранжа/ Ньютона с равномерной сеткой."""
    for ind, (function, interval, data) in enumerate(zip(functions, intervals, values)):
        print(f"Test: {ind + 1}")
        for x in data:
            print(f"Абсолютная ошибка Лагранжа в точке {x}:", abs_error(interpolate(function, x, *interval), function(x)))
            print(f"Абсолютная ошибка Ньютона в точке {x}:", abs_error(interpolate(function, x, *interval, is_Lagrange=False), function(x)))
            print("---")
        print()
        

def test_non_uniform_grid() -> None:
    """Тестирование интерполяции Лагранжа/ Ньютона с неравномерной сеткой."""
    grid = [0, 0.005, 0.0123, 0.051, 0,545, 0.675, 0.782, 0.895, 0.931, 1.012, 1.213, 1.324, 1.451, 1.567, 1.892, 2.123]
    
    for ind, (function, data) in enumerate(zip(functions, values)):
        print(f"Test: {ind + 1}")
        for x in data:
            print(f"Абсолютная ошибка Лагранжа в точке {x}:", abs_error(interpolate(function, x, grid=grid), function(x)))
            print(f"Абсолютная ошибка Ньютона в точке {x}:", abs_error(interpolate(function, x, grid=grid, is_Lagrange=False), function(x)))
            print("---")
        print()


def test_interpolation_evalution() -> None:
    """Тестирование ошиьки интерполяции Лагранжа/ Ньютона."""
    low_bound, high_bound = [], []
    interpolate_error_lagrange = []
    interpolate_error_newton = []
    
    data = values[3]
    degrees = []
    interval = intervals[3]
    function = functions[3]
    min_max_fun = [lambda x: max(x), lambda x: min(x)]
    
    for n, derivative_function, (a, b) in zip([1, 2], [lambda x: 2 - 0.5 * exp(x), lambda x: - 0.5 * exp(x)], [min_max_fun, min_max_fun]):
        for x in data:
            h = (interval[1] - interval[0]) / 11
            grid = [interval[0] + i * h for i in range(12)]
            low, high, lagrange, newton = get_interpolation_error_bounds(function, derivative_function, a, b, grid, x, n)
            
            low_bound.append(low)
            high_bound.append(high)
            degrees.append(n)
            interpolate_error_lagrange.append(function(x) - lagrange)
            interpolate_error_newton.append(function(x) - newton)
    
    result = pd.DataFrame({"x": data * 2, 
                           "Степень полинома": degrees, 
                           "Нижняя граница": low_bound,
                           "Ошибка интерполяции Лагранжа": interpolate_error_lagrange,
                           "Ошибка интерполяции Ньютона": interpolate_error_newton,
                           "Верхняя граница": high_bound})
    print(result)
        

if __name__ == "__main__":
    test_uniform_grid()
    test_non_uniform_grid()
    test_interpolation_evalution()
    

Test: 1
Абсолютная ошибка Лагранжа в точке 0.52: 1.296876883660758e-09
Абсолютная ошибка Ньютона в точке 0.52: 1.2968768281496068e-09
---
Абсолютная ошибка Лагранжа в точке 0.42: 3.859756403379322e-08
Абсолютная ошибка Ньютона в точке 0.42: 3.859756370072631e-08
---
Абсолютная ошибка Лагранжа в точке 0.87: 1.2597962451010858e-08
Абсолютная ошибка Ньютона в точке 0.87: 1.2597962339988555e-08
---
Абсолютная ошибка Лагранжа в точке 0.67: 2.9586439548312526e-10
Абсолютная ошибка Ньютона в точке 0.67: 2.958644787498521e-10
---

Test: 2
Абсолютная ошибка Лагранжа в точке 0.53: 1.2073675392798577e-15
Абсолютная ошибка Ньютона в точке 0.53: 1.0824674490095276e-15
---
Абсолютная ошибка Лагранжа в точке 0.52: 1.0824674490095276e-15
Абсолютная ошибка Ньютона в точке 0.52: 1.5265566588595902e-15
---
Абсолютная ошибка Лагранжа в точке 0.97: 1.5543122344752192e-15
Абсолютная ошибка Ньютона в точке 0.97: 7.216449660063518e-16
---
Абсолютная ошибка Лагранжа в точке 0.73: 1.3877787807814457e-17
Абсолют